### Step 1 - Start Spark Session, include additional configurations and common functions, and read data from the spark-warehouse database

In [ ]:
%run "../includes/configurations"

In [ ]:
%run "../includes/common_functions"

In [ ]:
from pyspark.sql.functions import concat, lit, to_timestamp

appName = "Drivers Ingestion"
spark = create_spark_context(appName, warehouse_location)

In [ ]:
drives_df = spark.read.parquet(f"{warehouse_location}/f1_raw.db/drivers")

### Step 2 - Rename and drop columns, and add new columns

In [ ]:
drives_column_mapping = {
    "driverId": "driver_id",
    "driverRef": "driver_ref"
}

drives_df = rename_columns(drives_df, drives_column_mapping)


In [ ]:
columns_to_drop = ['url']

drives_df = drop_columns(drives_df, columns_to_drop)

In [ ]:
drives_df = add_ingestion_date(drives_df)
drives_df = drives_df.withColumn("name", concat(
        drives_df["name.forename"],
        lit(" "),
        drives_df["name.surname"]
    )
)

### Step 3 - Write a data to a Datalake as a Parquet

In [ ]:
drives_df.write.mode("overwrite").parquet(f"{processed_folder_path}/drives")